# Tool 1 - Entity to Candidate Mapper (Parallel Multi-Agent Pattern)

**Status:** ✅ Ready for Databricks | **LLM Cost:** ~$0.003 per run | **Performance:** ~10s

**Pattern:** 2 Pydantic AI agents running in parallel via `asyncio.gather`

**Showcase:** Parallel multi-agent execution - both agents run simultaneously, results merged with consistency validation.

**Key Features:**
- 2 specialized agents: `ranking_agent` (top 10 candidates) + `mapping_agent` (1:1 entity mappings)
- Parallel execution with `asyncio.gather` (both agents run simultaneously)
- Consistency check validates overlap between ranked and mapped candidates (target: >70%)
- Expected performance: ~10s for parallel execution

**TODO:**
- [ ] Add retry logic for LLM failures (timeout, rate limit)
- [ ] Cache ranking results for repeated entity sets
- [ ] Test with 20+ entities (current test: 5-10)
- [ ] Add metadata quality score (completeness, freshness)

**IDEA:**
- Consider hybrid: embedding similarity for initial filtering + LLM for final ranking
- Add configurable top-N parameter (currently hardcoded to 10)
- Export consistency report as separate artifact

In [ ]:
# Install dependencies
%pip install pydantic-ai>=0.0.49 pydantic>=2.8.0

In [ ]:
# Restart Python kernel to use new packages
dbutils.library.restartPython()  # type: ignore

In [ ]:
import asyncio
import json
import os
from pydantic import BaseModel, Field
from pydantic_ai import Agent  # type: ignore

In [ ]:
# Configure Azure OpenAI from Databricks secrets
AZURE_ENDPOINT = dbutils.secrets.get(scope="mcop", key="azure-openai-endpoint").strip()  # type: ignore
AZURE_API_KEY = dbutils.secrets.get(scope="mcop", key="azure-openai-api-key").strip()  # type: ignore
DEPLOYMENT_NAME = dbutils.secrets.get(scope="mcop", key="azure-openai-deployment-name").strip()  # type: ignore

# Clean endpoint (remove /openai/v1/ if present - Pydantic AI will handle routing)
azure_endpoint_clean = AZURE_ENDPOINT.replace("/openai/v1/", "").replace("/openai/v1", "").rstrip("/")

# Set environment variables for Pydantic AI (Azure OpenAI compatible)
os.environ["OPENAI_BASE_URL"] = f"{azure_endpoint_clean}/openai/deployments/{DEPLOYMENT_NAME}"
os.environ["OPENAI_API_KEY"] = AZURE_API_KEY

MODEL_NAME = f"openai:{DEPLOYMENT_NAME}"
print(f"✅ Configured model: {MODEL_NAME}")
print(f"   Base URL: {os.environ['OPENAI_BASE_URL']}")


In [ ]:
# ==========================================
# PYDANTIC SCHEMAS - 4 modely pro 2 agenty
# ==========================================
# Tool 1 používá 2 specializované LLM agenty, které běží PARALELNĚ:
# - ranking_agent → vrací CandidateRanking (top 10 kandidátů)
# - mapping_agent → vrací MappingSuggestions (1:1 mapování entity→kandidát)
#
# Každý agent potřebuje vlastní output schema.

class CandidateRank(BaseModel):
    """Jeden kandidát z rankingu (1 položka v top 10).

    Co obsahuje:
    - entity_id: ID z metadata katalogu (např. UUID z Collibra)
    - rank: Pořadí v žebříčku (1 = nejlepší, 10 = nejhorší)
    - relevance_score: Skóre relevance 0.0-1.0 (vyšší = relevantnější)
    - justification: Textové zdůvodnění, proč je kandidát relevantní

    Příklad:
    {
      "entity_id": "c9c9fdf4-27ed-479d-b03b-f8ccec7a9f91",
      "rank": 1,
      "relevance_score": 0.95,
      "justification": "Perfect match: dimv_supplier table in dm_ba_purchase schema"
    }
    """
    entity_id: str = Field(description="Entity ID")
    rank: int = Field(description="Rank position (1=best)")
    relevance_score: float = Field(description="Relevance 0-1")
    justification: str = Field(description="Why this candidate is relevant")

class CandidateRanking(BaseModel):
    """Výstup z ranking_agent - top 10 kandidátů seřazených podle relevance.

    Toto je ROOT MODEL pro ranking_agent (result_type=CandidateRanking).
    LLM MUSÍ vrátit objekt s polem 'top_candidates' obsahujícím array 10 položek.

    Jak to funguje:
    1. ranking_agent dostane business entity + seznam ALL kandidátů z metadatu
    2. LLM ohodnotí kandidáty podle sémantické podobnosti, kvality, metadat
    3. Vrátí top 10 s pořadím, skóre a zdůvodněním
    """
    top_candidates: list[CandidateRank] = Field(description="Top 10 candidates")

class EntityMapping(BaseModel):
    """Jeden mapping: business entita → best matching candidate.

    Co obsahuje:
    - entity_name: Jméno business entity z Tool 0 (např. "Supplier Master")
    - matched_candidate_id: ID kandidáta z metadatu (UUID)
    - confidence: Jistota mapování 0.0-1.0 (>0.7 = high confidence)
    - reasoning: Textové vysvětlení, proč byl vybrán tento kandidát

    Příklad:
    {
      "entity_name": "Supplier Master",
      "matched_candidate_id": "c9c9fdf4-27ed-479d-b03b-f8ccec7a9f91",
      "confidence": 0.92,
      "reasoning": "Name match + description alignment + quality metadata present"
    }
    """
    entity_name: str = Field(description="Business entity name")
    matched_candidate_id: str = Field(description="Mapped candidate ID")
    confidence: float = Field(description="Mapping confidence 0-1")
    reasoning: str = Field(description="Mapping rationale")

class MappingSuggestions(BaseModel):
    """Výstup z mapping_agent - 1:1 mapování všech entit.

    Toto je ROOT MODEL pro mapping_agent (result_type=MappingSuggestions).
    LLM MUSÍ vrátit objekt s polem 'mappings' - array EntityMapping objektů.

    Jak to funguje:
    1. mapping_agent dostane stejný input jako ranking_agent
    2. LLM pro KAŽDOU business entitu vybere JEDNOHO nejlepšího kandidáta
    3. Vrátí array s N mapováními (N = počet business entit)

    Rozdíl oproti ranking:
    - Ranking: "Jaké jsou top 10 kandidáti celkově?"
    - Mapping: "Pro entitu X, který je nejlepší kandidát a proč?"
    """
    mappings: list[EntityMapping] = Field(description="Entity mappings")


In [ ]:
# ==========================================
# PYDANTIC AI AGENTS - 2 specializované LLM agenty
# ==========================================
# Pydantic AI pattern:
# Agent(model_name, result_type=Schema, system_prompt="...")
#
# Co to dělá:
# - Vytvoří LLM agenta s předem definovaným output schématem
# - result_type říká: "LLM MUSÍ vrátit data podle tohoto Pydantic modelu"
# - Pydantic AI automaticky:
#   1. Přidá schema do LLM promptu (JSON Schema format)
#   2. Zavolá LLM s function calling / JSON mode
#   3. Validuje response pomocí Pydantic
#   4. Vrátí type-safe Pydantic objekt (ne raw JSON string)

# ==========================================
# AGENT 1: ranking_agent
# ==========================================
# Účel: Ohodnotit všechny kandidáty a vrátit top 10
# Input: business_entities + technical_metadata
# Output: CandidateRanking (top 10 s rank, score, justification)
#
# System prompt vysvětluje:
# - Co má agent dělat (rank candidates)
# - Jaká kritéria použít (semantic similarity, data quality, metadata presence)
# - Jak formátovat output (structured ranking)
ranking_agent = Agent(
    MODEL_NAME,                      # "openai:test-gpt-5-mini" (Pydantic AI format)
    result_type=CandidateRanking,    # LLM MUSÍ vrátit CandidateRanking object
    system_prompt="""You are a data catalog expert.

Rank candidates by relevance to business entities:
- Consider semantic similarity (name, description)
- Evaluate data quality signals (completeness, freshness)
- Prioritize candidates with metadata (owner, lineage, tags)
- Top 10 most relevant candidates

Return structured ranking with scores and justifications."""
)

# ==========================================
# AGENT 2: mapping_agent
# ==========================================
# Účel: Pro každou entitu najít 1 nejlepší kandidát
# Input: business_entities + technical_metadata (STEJNÝ jako ranking_agent)
# Output: MappingSuggestions (N mappings, 1 per entity)
#
# Rozdíl oproti ranking_agent:
# - Ranking: "Top 10 candidates overall" (aggregovaný pohled)
# - Mapping: "Best match PER entity" (detailní 1:1 mapování)
#
# System prompt specifikuje:
# - 1:1 mapping (one entity → one candidate)
# - High confidence preferred (>0.7)
# - Clear reasoning required
mapping_agent = Agent(
    MODEL_NAME,
    result_type=MappingSuggestions,  # LLM MUSÍ vrátit MappingSuggestions object
    system_prompt="""You are a metadata mapping specialist.

Map each business entity to the BEST matching candidate:
- 1:1 mapping (one entity → one candidate)
- High confidence mappings (>0.7) preferred
- Consider semantic alignment and context
- Provide clear reasoning for each mapping

Return structured mappings with confidence scores."""
)

print("✅ Ranking and mapping agents created")


In [ ]:
# ==========================================
# PARALELNÍ SPUŠTĚNÍ 2 AGENTŮ - asyncio.gather()
# ==========================================
# Proč async pattern:
# - Oba agenti dostanou STEJNÝ input (entities + metadata)
# - Mají různé úkoly (ranking vs mapping)
# - Můžou běžet PARALELNĚ → šetří čas
# - asyncio.gather() počká na oba výsledky současně
#
# Klasický sequential:
# result1 = ranking_agent.run()   # ⏱️ 5s
# result2 = mapping_agent.run()   # ⏱️ 5s
# Total: 10s
#
# Paralel with asyncio.gather():
# results = asyncio.gather(ranking_agent.run(), mapping_agent.run())
# Total: ~5s (oba běží současně)

async def map_entities_to_candidates(
    business_entities: list[dict],     # Z Tool 0 (BusinessRequest.entities)
    technical_metadata: list[dict]     # Z katalogu (Collibra/Unity Catalog)
) -> dict:
    """
    Namapuje business entity na technické kandidáty pomocí 2 LLM agentů.

    Args:
        business_entities: Seznam entit z business požadavku
            Formát: [{"name": "Customer", "description": "...", "attributes": [...]}, ...]

        technical_metadata: Seznam technických kandidátů z katalogu
            Formát: [{"name": "dim_customer", "schema": "...", "description": "...", ...}, ...]

    Returns:
        dict s klíči:
        - "ranking": Top 10 candidates (CandidateRanking)
        - "mapping": 1:1 entity→candidate mappings (MappingSuggestions)
        - "consistency": Overlap analysis (kolik mapping kandidátů je v top 10)

    Pydantic AI workflow:
        1. Připrav input prompt (entities + metadata JSON)
        2. Spusť OBA agenty paralelně (asyncio.gather)
        3. Čekej na výsledky (result.data = Pydantic object)
        4. Validuj konzistenci (mapping by měl preferovat top 10 kandidáty)
        5. Vrať kombinované výsledky
    """

    # ==========================================
    # KROK 1: Připrav input data pro LLM
    # ==========================================
    # Pydantic AI automaticky přidá schema do promptu,
    # ale musíme dodat business kontext (entities + metadata)
    user_prompt = f"""
Business Entities (from requirement):
{json.dumps(business_entities, indent=2, ensure_ascii=False)}

Technical Metadata Candidates (from catalog):
{json.dumps(technical_metadata, indent=2, ensure_ascii=False)}
"""

    # ==========================================
    # KROK 2: Spusť OBA agenty PARALELNĚ
    # ==========================================
    # asyncio.gather() pattern:
    # - Vezme N async funkcí/coroutines
    # - Spustí všechny současně (non-blocking)
    # - Vrátí tuple s výsledky ve stejném pořadí
    #
    # agent.run_sync() vs agent.run():
    # - run_sync() = synchronní verze (pro non-async kód)
    # - run() = async verze (pro asyncio.gather)
    #
    # result object struktura:
    # - result.data = Pydantic object (CandidateRanking / MappingSuggestions)
    # - result.cost = API cost tracking (optional)
    # - result.timestamp = run timestamp

    ranking_result, mapping_result = await asyncio.gather(
        ranking_agent.run(user_prompt),      # Async coroutine 1
        mapping_agent.run(user_prompt)       # Async coroutine 2
    )

    # ==========================================
    # KROK 3: Extrahuj Pydantic objekty z result
    # ==========================================
    # result.data je už VALIDOVANÝ Pydantic object
    # (ne raw JSON string - to Pydantic AI už zpracoval)

    ranking_data: CandidateRanking = ranking_result.data
    # Type: CandidateRanking(root=[CandidateRank(...), ...])
    # Access: ranking_data.root[0].entity_id, ranking_data.root[0].rank, ...

    mapping_data: MappingSuggestions = mapping_result.data
    # Type: MappingSuggestions(root=[EntityMapping(...), ...])
    # Access: mapping_data.root[0].entity_name, mapping_data.root[0].matched_candidate_id, ...

    # ==========================================
    # KROK 4: Validuj konzistenci (optional check)
    # ==========================================
    # Dobrý mapping by měl preferovat kandidáty z top 10 rankingu.
    # Pokud mapping vrací úplně jiné kandidáty, může to signalizovat:
    # - Konfliktní kritéria mezi agenty
    # - Nedostatek kvalitních kandidátů
    # - Potřebu review člověkem

    top_10_ids = {candidate.entity_id for candidate in ranking_data.root}
    mapped_ids = {mapping.matched_candidate_id for mapping in mapping_data.root}
    overlap = top_10_ids.intersection(mapped_ids)

    consistency_note = (
        f"Mapping consistency: {len(overlap)}/{len(mapped_ids)} mapped candidates "
        f"are in top 10 ranking."
    )

    if len(overlap) < len(mapped_ids) * 0.5:  # Méně než 50% overlap
        consistency_note += " ⚠️ Low overlap - consider manual review."

    # ==========================================
    # KROK 5: Vrať kombinované výsledky
    # ==========================================
    return {
        "ranking": ranking_data.model_dump(),      # Convert Pydantic → dict
        "mapping": mapping_data.model_dump(),      # Convert Pydantic → dict
        "consistency": {
            "overlap_count": len(overlap),
            "total_mappings": len(mapped_ids),
            "overlap_percentage": len(overlap) / len(mapped_ids) if mapped_ids else 0,
            "note": consistency_note
        }
    }

print("✅ Entity mapping function defined (async with parallel agents)")

In [ ]:
# Load input data from DBFS
tool0_path = "/dbfs/FileStore/mcop/tool0_samples/sample_business_request.json"
metadata_path = "/dbfs/FileStore/mcop/metadata/BA-BS_Datamarts_metadata.json"

with open(tool0_path, "r") as f:
    tool0_data = json.load(f)

with open(metadata_path, "r") as f:
    metadata = json.load(f)

entities = tool0_data.get("entities", [])
print(f"✅ Loaded {len(entities)} entities from Tool 0")
print(f"✅ Loaded {len(metadata)} metadata items")

In [ ]:
# Run parallel mapping
result = await map_entities_to_candidates(entities, metadata)

print(f"\n✅ Parallel mapping complete")
print(f"   Rankings: {len(result['rankings'])}")
print(f"   Mappings: {len(result['mappings'])}")
print(f"   Consistency: {result['consistency_check']['consistency_ratio']:.1%} ({result['consistency_check']['status']})")

In [ ]:
# Save results to DBFS
output_path = "/dbfs/FileStore/mcop/tool1/filtered_dataset.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(result, f, indent=2)

print(f"✅ Results saved: {output_path}")

In [ ]:
# Display sample results with consistency check
print("\n" + "="*80)
print("PARALLEL MULTI-AGENT MAPPING RESULTS")
print("="*80)

print(f"\n🏆 Top 5 Ranked Candidates:")
for candidate in result['rankings'][:5]:
    print(f"   {candidate['rank']}. {candidate['entity_id']} (score: {candidate['relevance_score']:.2f})")
    print(f"      → {candidate['justification'][:80]}...")

print(f"\n🔗 Sample Mappings (top 5):")
for mapping in result['mappings'][:5]:
    print(f"   {mapping['entity_name']} → {mapping['matched_candidate_id']}")
    print(f"      Confidence: {mapping['confidence']:.1%}")
    print(f"      Reasoning: {mapping['reasoning'][:80]}...")

print(f"\n✅ Consistency Check:")
print(f"   Overlap: {result['consistency_check']['overlap_count']} / {result['consistency_check']['total_mappings']}")
print(f"   Ratio: {result['consistency_check']['consistency_ratio']:.1%}")
print(f"   Status: {result['consistency_check']['status']}")
print(f"   Expected: >70% for good quality (ranking and mapping agree)")

print("="*80)